# FVCOM Member-Node Mapping Demo
**Author: Jun Sasaki  Coded on 2025-10-16**<br>

This notebook demonstrates how to extract and visualize member-to-node mappings from FVCOM ensemble dye runs.

## Features
- Parse FVCOM namelist files to extract dye source configurations
- Identify which nodes are active in each ensemble member
- Extract node coordinates from NetCDF output files
- Visualize member-node mappings in tables and maps

## 1. Setup and Imports

In [ ]:
import sys
from pathlib import Path

# Add xfvcom to path
xfvcom_root = Path.cwd().parents[1]
sys.path.insert(0, str(xfvcom_root))

import pandas as pd
import matplotlib.pyplot as plt

from xfvcom.ensemble_analysis.member_info import (
    extract_member_node_mapping,
    get_member_summary,
    get_node_coordinates,
    export_member_mapping,
)
from xfvcom.io.nml_parser import NamelistParser

print("✓ Imports successful")
print(f"xfvcom root: {xfvcom_root}")

## 2. Configuration

In [ ]:
# Path configuration
tb_fvcom_dir = Path("~/Github/TB-FVCOM").expanduser()
nml_dir = tb_fvcom_dir / "goto2023/dye_run"
output_dir = tb_fvcom_dir / "goto2023/dye_run/output/2021"

# Case configuration
basename = "tb_w18_r16"
year = 2021
members = list(range(19))  # Members 0-18

print(f"TB-FVCOM directory: {tb_fvcom_dir}")
print(f"Namelist directory: {nml_dir}")
print(f"Output directory: {output_dir}")
print(f"Configuration: {basename}, year {year}, members {members}")

## 3. Extract Member-Node Mapping

Parse all namelist files to extract which nodes are active in each member.

In [ ]:
# Extract full member-node mapping
mapping_df = extract_member_node_mapping(
    nml_dir=nml_dir,
    basename=basename,
    year=year,
    members=members,
)

print(f"Extracted mapping for {len(mapping_df)} source-member combinations")
print(f"\nDataFrame shape: {mapping_df.shape}")
print(f"Columns: {list(mapping_df.columns)}")
mapping_df.head(10)

## 4. Member Summary

Get a summary of active sources for each member.

In [ ]:
# Get member summary
summary_df = get_member_summary(
    nml_dir=nml_dir,
    basename=basename,
    year=year,
    members=members,
)

print("Member Summary:")
print("=" * 80)
# Display without truncation
pd.set_option('display.max_colwidth', None)
summary_df

## 5. Examine Specific Members

Look at detailed configuration for specific members.

In [ ]:
# Member 0: All sources (reference)
print("Member 0 (All sources):")
print("=" * 60)
member_0 = mapping_df[mapping_df['member'] == 0]
print(f"Total sources: {len(member_0)}")
print(f"Total dye release rate: {member_0['strength'].sum():.2f}")
print("\nSources:")
member_0[['source_name', 'node_id', 'strength', 'source_type']]

In [ ]:
# Member 1: Arakawa group
print("Member 1 (Arakawa group):")
print("=" * 60)
member_1 = mapping_df[mapping_df['member'] == 1]
print(f"Total sources: {len(member_1)}")
print(f"Total dye release rate: {member_1['strength'].sum():.2f}")
print("\nSources:")
member_1[['source_name', 'node_id', 'strength', 'source_type']]

In [ ]:
# Member 2: Sumidagawa group
print("Member 2 (Sumidagawa group):")
print("=" * 60)
member_2 = mapping_df[mapping_df['member'] == 2]
print(f"Total sources: {len(member_2)}")
print(f"Total dye release rate: {member_2['strength'].sum():.2f}")
print("\nSources:")
member_2[['source_name', 'node_id', 'strength', 'source_type']]

## 6. Extract Node Coordinates

Get geographic coordinates for all active nodes from a sample NetCDF file.

In [ ]:
# Get unique node IDs across all members
unique_nodes = mapping_df['node_id'].unique()
print(f"Total unique nodes across all members: {len(unique_nodes)}")
print(f"Node IDs: {sorted(unique_nodes)}")

# Sample NetCDF file (use member 0 or 1)
sample_nc = output_dir / "0" / f"{basename}_2021_0_0001.nc"

if sample_nc.exists():
    # Extract coordinates
    coords_df = get_node_coordinates(sample_nc, unique_nodes.tolist())
    
    print(f"\nExtracted coordinates for {len(coords_df)} nodes")
    coords_df.head(10)
else:
    print(f"Warning: Sample NetCDF file not found: {sample_nc}")
    print("Skipping coordinate extraction")

## 7. Merge Mapping with Coordinates

Combine member-node mapping with geographic coordinates.

In [ ]:
if 'coords_df' in locals():
    # Merge mapping with coordinates
    mapping_with_coords = mapping_df.merge(
        coords_df,
        on='node_id',
        how='left'
    )
    
    print(f"Merged DataFrame shape: {mapping_with_coords.shape}")
    print(f"Columns: {list(mapping_with_coords.columns)}")
    mapping_with_coords.head(10)
else:
    print("Skipping merge (coordinates not available)")

## 8. Visualize Node Locations

Plot the locations of dye release nodes on a map.

In [ ]:
if 'coords_df' in locals():
    # Create a simple scatter plot of node locations
    fig, ax = plt.subplots(figsize=(12, 8))
    
    # Plot all unique nodes
    ax.scatter(
        coords_df['lon'],
        coords_df['lat'],
        c='blue',
        s=100,
        alpha=0.6,
        edgecolors='black',
        linewidth=1
    )
    
    # Add node IDs as labels
    for _, row in coords_df.iterrows():
        ax.annotate(
            str(int(row['node_id'])),
            xy=(row['lon'], row['lat']),
            xytext=(5, 5),
            textcoords='offset points',
            fontsize=8,
            bbox=dict(boxstyle='round,pad=0.3', facecolor='white', alpha=0.7)
        )
    
    ax.set_xlabel('Longitude', fontsize=12)
    ax.set_ylabel('Latitude', fontsize=12)
    ax.set_title('Dye Release Node Locations', fontsize=14)
    ax.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()
    
    print(f"Plotted {len(coords_df)} dye release nodes")
else:
    print("Skipping visualization (coordinates not available)")

## 9. Source Type Analysis

Analyze the distribution of rivers vs sewers across members.

In [ ]:
# Count sources by type for each member
source_type_counts = mapping_df.groupby(['member', 'source_type']).size().unstack(fill_value=0)

print("Source Type Distribution by Member:")
print("=" * 60)
source_type_counts

In [ ]:
# Visualize source type distribution
fig, ax = plt.subplots(figsize=(12, 6))

source_type_counts.plot(
    kind='bar',
    ax=ax,
    color=['#1f77b4', '#ff7f0e'],
    width=0.8
)

ax.set_xlabel('Member', fontsize=12)
ax.set_ylabel('Number of Sources', fontsize=12)
ax.set_title('Number of Active Sources by Member and Type', fontsize=14)
ax.legend(title='Source Type', fontsize=10)
ax.grid(True, alpha=0.3, axis='y')

plt.tight_layout()
plt.show()

## 10. Export Results

Export member-node mappings to various formats for documentation.

In [ ]:
# Create output directory
export_dir = Path.cwd().parent / "output"
export_dir.mkdir(exist_ok=True)

print(f"Export directory: {export_dir}")

# Export full mapping to CSV
export_member_mapping(
    mapping_df,
    export_dir / "member_node_mapping.csv",
    format='csv'
)

# Export summary to CSV
summary_df.to_csv(export_dir / "member_summary.csv", index=False)
print(f"Exported to: {export_dir / 'member_summary.csv'}")

# Export to markdown
export_member_mapping(
    mapping_df,
    export_dir / "member_node_mapping.md",
    format='markdown'
)

# Export coordinates if available
if 'coords_df' in locals():
    coords_df.to_csv(export_dir / "node_coordinates.csv", index=False)
    print(f"Exported to: {export_dir / 'node_coordinates.csv'}")

print("\n✓ Export complete")

## 11. Summary

Review the key findings.

In [ ]:
print("MEMBER-NODE MAPPING SUMMARY")
print("=" * 80)
print(f"Configuration:")
print(f"  Case: {basename}")
print(f"  Year: {year}")
print(f"  Members analyzed: {len(members)}")
print()
print(f"Results:")
print(f"  Total unique nodes: {len(mapping_df['node_id'].unique())}")
print(f"  Total unique sources: {len(mapping_df['source_name'].unique())}")
print(f"  Source-member combinations: {len(mapping_df)}")
print()
print(f"Source breakdown:")
print(f"  Rivers: {len(mapping_df[mapping_df['source_type'] == 'River']['source_name'].unique())}")
print(f"  Sewers: {len(mapping_df[mapping_df['source_type'] == 'Sewer']['source_name'].unique())}")
print()
print(f"Member configuration patterns:")
for member in [0, 1, 2, 3, 4, 5]:
    member_data = summary_df[summary_df['member'] == member]
    if not member_data.empty:
        n_sources = member_data['n_sources'].iloc[0]
        sources = member_data['source_names'].iloc[0]
        print(f"  Member {member}: {n_sources} sources - {sources}")
print("=" * 80)